# Loading packages

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random
import gc

from os import path
from matplotlib import pyplot as plt
from tensorflow.keras import layers, losses, optimizers, metrics, callbacks, Model, Input, regularizers

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=9048)]
    )

logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

1 Physical GPU, 1 Logical GPUs


In [3]:
SEED = 123
N_CLASS = 10
IMG_SIZE = 32

In [4]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Loading datasets

In [5]:
dataset = np.load("resnet_output.npy")
dataset.shape

(50000, 2048)

In [6]:
filenames = list(os.walk(path.join(os.getcwd(), '..', 'cifar-10', 'train', 'train_images'), topdown=False))[0][2]
filenames = pd.DataFrame(filenames, columns=["name"]).reset_index(names="real_index")
filenames

,real_index,name
0,0,1.png
1,1,10.png
2,2,100.png
3,3,1000.png
4,4,10000.png
...,...,...
49995,49995,9995.png
49996,49996,9996.png
49997,49997,9997.png
49998,49998,9998.png


In [7]:
train_labels_df = pd.read_csv(path.join(os.getcwd(), '..', 'train_val_test', 'train.csv')).drop(["Unnamed: 0"], axis=1)
train_labels_df = train_labels_df.merge(filenames, on=["name"])
train_labels_df

,name,codes,real_index
0,1.png,6,0
1,100.png,1,2
2,10001.png,1,5
3,10002.png,6,6
4,10003.png,6,7
...,...,...,...
34995,9994.png,1,49994
34996,9996.png,3,49996
34997,9997.png,9,49997
34998,9998.png,1,49998


In [8]:
val_labels_df = pd.read_csv(path.join(os.getcwd(), '..', 'train_val_test', 'val.csv')).drop(["Unnamed: 0"], axis=1)
val_labels_df = val_labels_df.merge(filenames, on=["name"])
val_labels_df

,name,codes,real_index
0,10.png,3,1
1,1000.png,5,3
2,10000.png,5,4
3,10014.png,6,19
4,10029.png,7,35
...,...,...,...
4995,9961.png,8,49958
4996,9977.png,3,49975
4997,998.png,1,49978
4998,9986.png,0,49985


In [9]:
test_labels_df = pd.read_csv(path.join(os.getcwd(), '..', 'train_val_test', 'test.csv')).drop(["Unnamed: 0"], axis=1)
test_labels_df = test_labels_df.merge(filenames, on=["name"])
test_labels_df

,name,codes,real_index
0,10008.png,6,12
1,1001.png,9,14
2,10012.png,3,17
3,10015.png,5,20
4,10021.png,0,27
...,...,...,...
9995,9971.png,3,49969
9996,9981.png,6,49980
9997,9983.png,3,49982
9998,9984.png,0,49983


In [10]:
assert not train_labels_df.real_index.isin(test_labels_df).any()
assert not train_labels_df.real_index.isin(val_labels_df).any()
assert not test_labels_df.real_index.isin(val_labels_df).any()

In [11]:
train_x, train_y = dataset[train_labels_df.real_index, :], tf.one_hot(train_labels_df.codes, N_CLASS)
val_x, val_y = dataset[val_labels_df.real_index, :], tf.one_hot(val_labels_df.codes, N_CLASS)
test_x, test_y = dataset[test_labels_df.real_index, :], tf.one_hot(test_labels_df.codes, N_CLASS)
train_x.shape, val_x.shape, test_x.shape

((35000, 2048), (5000, 2048), (10000, 2048))

# Resnet FC

# Regularization

Training will be repeated 10 times with different weights initialization.

In [12]:
def create_model(regularizer=None):
    input_layer = Input(shape=(2048))
    x = layers.Dense(2048, activation="relu", kernel_regularizer=regularizer)(input_layer)
    x = layers.BatchNormalization()(x)
    output = layers.Dense(N_CLASS, kernel_regularizer=regularizer, activation="softmax")(x)
    model = Model(inputs=input_layer, outputs=output)
    return model

model = create_model(regularizer=regularizers.L1())
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2048)]            0         
                                                                 
 dense (Dense)               (None, 2048)              4196352   
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 10)                20490     
                                                                 
Total params: 4,225,034
Trainable params: 4,220,938
Non-trainable params: 4,096
_________________________________________________________________


In [13]:
regularizers_ = [
    regularizers.L1(),
    regularizers.L2(),
    regularizers.L1L2()
]

early_stopping = callbacks.EarlyStopping(
    monitor='val_categorical_accuracy',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5, min_lr=0.00001, verbose=1)

In [14]:
TRAINING_SEEDS = list(range(10))
results = []
for regularizer in regularizers_:
    for seed in TRAINING_SEEDS:
        random.seed(seed)
        np.random.seed(SEED)
        tf.random.set_seed(seed)

        model = create_model(regularizer=regularizer)

        model.compile(
            optimizer=optimizers.Adam(learning_rate=0.001),
            loss=losses.CategoricalCrossentropy(),
            metrics=[metrics.CategoricalAccuracy(), metrics.CategoricalCrossentropy()]
        )

        history = model.fit(
            x=train_x, y=train_y,
            epochs=200,
            batch_size=512,
            validation_data=(val_x, val_y),
            shuffle=True,
            callbacks=[early_stopping, reduce_lr],
            verbose=0
        )
        eval_results = model.evaluate(x=test_x, batch_size=512, y=test_y)

        results += [{
            'seed': seed,
            'regularizer': regularizer.__class__.__name__,
            'results': dict(zip(model.metrics_names, eval_results))
        }]
        gc.collect()

results = pd.DataFrame(results)
results = pd.concat([results.drop(["results"], axis=1), results["results"].apply(pd.Series)], axis=1)
results.to_csv('l1_l2_results.csv')


Epoch 9: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 20: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 27: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 35: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 40: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
20/20 [==============================] - 0s 4ms/step - loss: 1.2801 - categorical_accuracy: 0.8928 - categorical_crossentropy: 0.3527

Epoch 8: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 14: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 23: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 37: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 44: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 50: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 61: 

In [15]:
results

,seed,regularizer,loss,categorical_accuracy,categorical_crossentropy
0,0,L1,1.280071,0.8928,0.352705
1,1,L1,0.602628,0.9100,0.305946
2,2,L1,5.522645,0.8409,0.477656
3,3,L1,0.583234,0.9113,0.288536
4,4,L1,0.588320,0.9083,0.304202
5,5,L1,0.592329,0.9109,0.290358
6,6,L1,0.598997,0.9082,0.296166
7,7,L1,0.572063,0.9084,0.299819
8,8,L1,3.208997,0.8731,0.410642
9,9,L1,0.593729,0.9110,0.292527


# Dropout

In [21]:
def create_model(dropout_val=None):
    input_layer = Input(shape=(2048))
    x = layers.Dropout(dropout_val)(input_layer)
    x = layers.Dense(2048, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_val)(x)
    output = layers.Dense(N_CLASS, activation="softmax")(x)
    model = Model(inputs=input_layer, outputs=output)
    return model

model = create_model(0.1)
model.summary()

Model: "model_72"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_74 (InputLayer)       [(None, 2048)]            0         
                                                                 
 dropout_83 (Dropout)        (None, 2048)              0         
                                                                 
 dense_144 (Dense)           (None, 2048)              4196352   
                                                                 
 batch_normalization_72 (Bat  (None, 2048)             8192      
 chNormalization)                                                
                                                                 
 dropout_84 (Dropout)        (None, 2048)              0         
                                                                 
 dense_145 (Dense)           (None, 10)                20490     
                                                          

In [22]:
dropout_values = [0.1, 0.2, 0.3, 0.5]

In [23]:
TRAINING_SEEDS = list(range(10))
results = []

for rate in dropout_values:
    for seed in TRAINING_SEEDS:
        random.seed(seed)
        np.random.seed(SEED)
        tf.random.set_seed(seed)

        model = create_model(rate)

        model.compile(
            optimizer=optimizers.Adam(learning_rate=0.001),
            loss=losses.CategoricalCrossentropy(),
            metrics=[metrics.CategoricalAccuracy(), metrics.CategoricalCrossentropy()]
        )

        history = model.fit(
            x=train_x, y=train_y,
            epochs=200,
            batch_size=512,
            validation_data=(val_x, val_y),
            shuffle=True,
            callbacks=[early_stopping, reduce_lr],
            verbose=0
        )
        
        eval_results = model.evaluate(x=test_x, batch_size=512, y=test_y)

        results += [{
            'seed': seed,
            'rate': rate,
            'results': dict(zip(model.metrics_names, eval_results))
        }]
        
        gc.collect()

results = pd.DataFrame(results)
results = pd.concat([results.drop(["results"], axis=1), results["results"].apply(pd.Series)], axis=1)
results.to_csv('dropout_results.csv')


Epoch 15: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 29: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 36: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 41: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
20/20 [==============================] - 0s 3ms/step - loss: 0.3041 - categorical_accuracy: 0.9270 - categorical_crossentropy: 0.3041

Epoch 16: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 30: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 35: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
20/20 [==============================] - 0s 4ms/step - loss: 0.3013 - categorical_accuracy: 0.9251 - categorical_crossentropy: 0.3013

Epoch 13: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 30: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 35: ReduceLROnPlateau re

In [24]:
results

,seed,rate,loss,categorical_accuracy,categorical_crossentropy
0,0,0.1,0.304116,0.9270,0.304116
1,1,0.1,0.301261,0.9251,0.301261
2,2,0.1,0.315159,0.9230,0.315159
3,3,0.1,0.300282,0.9290,0.300282
4,4,0.1,0.310049,0.9282,0.310049
5,5,0.1,0.272612,0.9254,0.272612
6,6,0.1,0.270459,0.9240,0.270459
7,7,0.1,0.299823,0.9299,0.299823
8,8,0.1,0.314256,0.9285,0.314256
9,9,0.1,0.312211,0.9261,0.312211
